# Chapter 6: Always Blocks and Processes

## Introduction

Always blocks are fundamental constructs in SystemVerilog that describe how hardware behaves over time. They define processes that execute continuously during simulation and represent different types of hardware structures. SystemVerilog provides three specialized always blocks that make design intent clearer and help avoid common modeling mistakes.

## Types of Always Blocks

SystemVerilog introduces three specialized always blocks:

- `always_comb` - For combinational logic
- `always_ff` - For sequential logic (flip-flops)
- `always_latch` - For latches

These replace the generic `always` block from Verilog and provide better checking and clearer intent.

### always_comb for Combinational Logic

The `always_comb` block is used to model combinational logic where outputs change immediately when inputs change.

#### Basic Syntax

```systemverilog
always_comb begin
    // Combinational logic statements
end
```

#### Key Features

- Automatically sensitive to all inputs (no sensitivity list needed)
- Executes immediately when any input changes
- Should not contain clocked logic or memory elements
- Helps catch incomplete sensitivity lists

#### Examples

##### Simple Multiplexer
```systemverilog
module mux2to1 (
    input  logic sel,
    input  logic a, b,
    output logic y
);

always_comb begin
    if (sel)
        y = b;
    else
        y = a;
end

endmodule
```

##### ALU Example
```systemverilog
module simple_alu (
    input  logic [3:0] a, b,
    input  logic [1:0] op,
    output logic [3:0] result,
    output logic       zero
);

always_comb begin
    case (op)
        2'b00: result = a + b;    // Add
        2'b01: result = a - b;    // Subtract
        2'b10: result = a & b;    // AND
        2'b11: result = a | b;    // OR
        default: result = 4'b0;
    endcase
    
    zero = (result == 4'b0);
end

endmodule
```

##### Priority Encoder
```systemverilog
module priority_encoder (
    input  logic [7:0] req,
    output logic [2:0] grant,
    output logic       valid
);

always_comb begin
    valid = |req;  // OR reduction - true if any bit set
    
    if (req[7])      grant = 3'd7;
    else if (req[6]) grant = 3'd6;
    else if (req[5]) grant = 3'd5;
    else if (req[4]) grant = 3'd4;
    else if (req[3]) grant = 3'd3;
    else if (req[2]) grant = 3'd2;
    else if (req[1]) grant = 3'd1;
    else if (req[0]) grant = 3'd0;
    else             grant = 3'd0;
end

endmodule
```

### always_ff for Sequential Logic

The `always_ff` block is used to model sequential logic elements like flip-flops and registers.

#### Basic Syntax

```systemverilog
always_ff @(posedge clk) begin
    // Sequential logic statements
end

// With reset
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        // Reset logic
    end else begin
        // Normal clocked logic
    end
end
```

#### Key Features

- Must have a clocking event in sensitivity list
- Models memory elements (flip-flops, registers)
- Can include asynchronous reset/set
- Should use non-blocking assignments (<=) for clocked logic

#### Examples

##### Simple D Flip-Flop
```systemverilog
module dff (
    input  logic clk,
    input  logic rst_n,
    input  logic d,
    output logic q
);

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        q <= 1'b0;
    else
        q <= d;
end

endmodule
```

##### Counter with Enable
```systemverilog
module counter (
    input  logic       clk,
    input  logic       rst_n,
    input  logic       enable,
    input  logic       load,
    input  logic [7:0] load_value,
    output logic [7:0] count
);

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        count <= 8'b0;
    end else if (load) begin
        count <= load_value;
    end else if (enable) begin
        count <= count + 1;
    end
    // If neither load nor enable, count maintains its value
end

endmodule
```

##### Shift Register
```systemverilog
module shift_register #(
    parameter WIDTH = 8
)(
    input  logic             clk,
    input  logic             rst_n,
    input  logic             shift_en,
    input  logic             serial_in,
    output logic             serial_out,
    output logic [WIDTH-1:0] parallel_out
);

logic [WIDTH-1:0] shift_reg;

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        shift_reg <= '0;
    end else if (shift_en) begin
        shift_reg <= {shift_reg[WIDTH-2:0], serial_in};
    end
end

assign serial_out = shift_reg[WIDTH-1];
assign parallel_out = shift_reg;

endmodule
```

##### State Machine Example
```systemverilog
typedef enum logic [1:0] {
    IDLE = 2'b00,
    ACTIVE = 2'b01,
    WAIT = 2'b10,
    DONE = 2'b11
} state_t;

module fsm (
    input  logic   clk,
    input  logic   rst_n,
    input  logic   start,
    input  logic   ready,
    output logic   busy,
    output logic   complete
);

state_t current_state, next_state;

// State register
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        current_state <= IDLE;
    else
        current_state <= next_state;
end

// Next state logic (combinational)
always_comb begin
    case (current_state)
        IDLE: begin
            if (start)
                next_state = ACTIVE;
            else
                next_state = IDLE;
        end
        
        ACTIVE: begin
            if (ready)
                next_state = WAIT;
            else
                next_state = ACTIVE;
        end
        
        WAIT: begin
            next_state = DONE;
        end
        
        DONE: begin
            next_state = IDLE;
        end
        
        default: next_state = IDLE;
    endcase
end

// Output logic (combinational)
always_comb begin
    busy = (current_state != IDLE) && (current_state != DONE);
    complete = (current_state == DONE);
end

endmodule
```

### always_latch for Latches

The `always_latch` block is used to model transparent latches, though latches are generally discouraged in synchronous design.

#### Basic Syntax

```systemverilog
always_latch begin
    // Latch logic statements
end
```

#### Key Features

- Models level-sensitive storage elements
- Should be avoided in most synchronous designs
- Can cause timing issues and make verification difficult
- Sometimes used for specific analog or mixed-signal applications

#### Example

```systemverilog
module d_latch (
    input  logic enable,
    input  logic d,
    output logic q
);

always_latch begin
    if (enable)
        q = d;
    // When enable is low, q retains its value (latch behavior)
end

endmodule
```

### Blocking vs. Non-Blocking Assignments

Understanding the difference between blocking (=) and non-blocking (<=) assignments is crucial for proper hardware modeling.

#### Blocking Assignments (=)

- Execute immediately in sequence
- Used in combinational logic (`always_comb`)
- Model wire-like behavior
- Can create race conditions if misused

#### Non-Blocking Assignments (<=)

- Scheduled to execute at end of time step
- Used in sequential logic (`always_ff`)
- Model register-like behavior
- Prevent race conditions in clocked logic

#### Examples Comparing Both

##### Combinational Logic - Use Blocking (=)
```systemverilog
// Correct - using blocking assignments
always_comb begin
    temp = a & b;
    y = temp | c;
end

// Incorrect - non-blocking in combinational logic
always_comb begin
    temp <= a & b;  // Wrong!
    y <= temp | c;  // Wrong! - temp not updated yet
end
```

##### Sequential Logic - Use Non-Blocking (<=)
```systemverilog
// Correct - using non-blocking assignments
always_ff @(posedge clk) begin
    q1 <= d;
    q2 <= q1;  // Creates shift register
end

// Incorrect - blocking in sequential logic
always_ff @(posedge clk) begin
    q1 = d;
    q2 = q1;   // Both update to 'd' simultaneously - not a shift register!
end
```

### Best Practices Summary

1. **Combinational logic (`always_comb`)**: Use blocking assignments (=)
2. **Sequential logic (`always_ff`)**: Use non-blocking assignments (<=)
3. **Mixed assignments**: Never mix blocking and non-blocking in the same always block

### Race Conditions and Common Pitfalls

#### Race Condition Example

```systemverilog
// Problematic code - race condition
module race_example (
    input  logic clk,
    input  logic d,
    output logic q1, q2
);

// Two separate always blocks updating at same time
always_ff @(posedge clk) begin
    q1 <= d;
end

always_ff @(posedge clk) begin
    q2 <= q1;  // Race condition! Order of execution matters
end

endmodule
```

```systemverilog
// Better approach - combine into one always block
always_ff @(posedge clk) begin
    q1 <= d;
    q2 <= q1;  // Now guaranteed to work correctly
end
```

#### Incomplete Sensitivity Lists (Verilog issue fixed by always_comb)

```systemverilog
// Old Verilog style - error prone
always @(a, b) begin  // Forgot 'c' in sensitivity list!
    y = a & b;
    z = y | c;        // 'z' won't update when 'c' changes
end

// SystemVerilog solution - automatic sensitivity
always_comb begin
    y = a & b;
    z = y | c;        // Automatically sensitive to a, b, and c
end
```

#### Process Control and Advanced Topics

#### Multiple Clock Domains

```systemverilog
module dual_clock_design (
    input  logic clk1, clk2,
    input  logic rst_n,
    input  logic data_in,
    output logic data_out
);

logic ff1, ff2;

// Clock domain 1
always_ff @(posedge clk1 or negedge rst_n) begin
    if (!rst_n)
        ff1 <= 1'b0;
    else
        ff1 <= data_in;
end

// Clock domain 2
always_ff @(posedge clk2 or negedge rst_n) begin
    if (!rst_n)
        ff2 <= 1'b0;
    else
        ff2 <= ff1;  // Potential metastability issue!
end

assign data_out = ff2;

endmodule
```

#### Generate Blocks with Always Blocks

```systemverilog
module parameterized_register #(
    parameter WIDTH = 8,
    parameter STAGES = 4
)(
    input  logic             clk,
    input  logic             rst_n,
    input  logic [WIDTH-1:0] data_in,
    output logic [WIDTH-1:0] data_out
);

logic [WIDTH-1:0] pipe_reg [STAGES-1:0];

genvar i;
generate
    for (i = 0; i < STAGES; i++) begin : pipe_stage
        always_ff @(posedge clk or negedge rst_n) begin
            if (!rst_n) begin
                pipe_reg[i] <= '0;
            end else begin
                if (i == 0)
                    pipe_reg[i] <= data_in;
                else
                    pipe_reg[i] <= pipe_reg[i-1];
            end
        end
    end
endgenerate

assign data_out = pipe_reg[STAGES-1];

endmodule
```

### Best Practices and Guidelines

#### Design Guidelines

1. **Use appropriate always block types**:
   - `always_comb` for combinational logic
   - `always_ff` for sequential logic
   - Avoid `always_latch` unless specifically needed

2. **Assignment types**:
   - Blocking (=) in `always_comb`
   - Non-blocking (<=) in `always_ff`
   - Never mix both types in the same block

3. **Reset strategy**:
   - Use asynchronous reset for critical paths
   - Consider synchronous reset for better timing
   - Initialize all registers consistently

4. **Avoid common mistakes**:
   - Incomplete case statements
   - Inferred latches from incomplete if-else chains
   - Multiple drivers to same signal

#### Code Style Example

```systemverilog
module good_style_example (
    input  logic       clk,
    input  logic       rst_n,
    input  logic [7:0] data_in,
    input  logic       enable,
    output logic [7:0] data_out,
    output logic       valid
);

// Sequential logic - use always_ff with non-blocking
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        data_out <= 8'b0;
        valid    <= 1'b0;
    end else if (enable) begin
        data_out <= data_in;
        valid    <= 1'b1;
    end else begin
        valid    <= 1'b0;
        // data_out retains its value
    end
end

endmodule
```

### Summary

Always blocks are the cornerstone of behavioral modeling in SystemVerilog. The three specialized types (`always_comb`, `always_ff`, `always_latch`) provide clear intent and help prevent common coding errors. Understanding when to use blocking vs. non-blocking assignments is crucial for creating hardware that behaves as intended. Following the best practices outlined in this chapter will lead to more reliable, synthesizable, and maintainable code.


### Key Takeaways

- Use `always_comb` for combinational logic with blocking assignments (=)
- Use `always_ff` for sequential logic with non-blocking assignments (<=)
- Avoid `always_latch` unless specifically required
- Be consistent with reset strategies
- Understand race conditions and how to avoid them
- Always consider the hardware implications of your code

# SystemVerilog Always Blocks and Processes - Example Index

## always_comb for Combinational Logic

#### Basic Logic Gates
Simple AND, OR, XOR, NAND gates demonstrating basic combinational behavior

In [6]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_1__basic_logic_gates/"
files = ["basic_logic_gates.sv", "basic_logic_gates_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// basic_logic_gates.sv
module basic_logic_gates (
    input  logic a,        // First input
    input  logic b,        // Second input
    output logic and_out,  // AND gate output
    output logic or_out,   // OR gate output  
    output logic xor_out,  // XOR gate output
    output logic nand_out  // NAND gate output
);

    // Combinational logic assignments
    assign and_out  = a & b;   // AND gate
    assign or_out   = a | b;   // OR gate
    assign xor_out  = a ^ b;   // XOR gate
    assign nand_out = ~(a & b); // NAND gate (inverted AND)

    // Display gate outputs whenever inputs change
    always_comb begin
        $display(
            "Inputs: a=%b, b=%b | Outputs: AND=%b, OR=%b, XOR=%b, NAND=%b", 
            a, b, and_out, or_out, xor_out, nand_out);
    end

endmodule
```

```systemverilog
// basic_logic_gates_testbench.sv
module logic_gates_testbench;

    // Test signals
    logic test_a, test_b;
    logic result_and, result_or, result_xor, result_nand;

    // Instantiate the design under test
    basic_logic_gates DUT (
        .a(test_a),
        .b(test_b),
        .and_out(result_and),
        .or_out(result_or),
        .xor_out(result_xor),
        .nand_out(result_nand)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("basic_logic_gates_testbench.vcd");
        $dumpvars(0, basic_logic_gates_testbench);
        
        $display("=== Basic Logic Gates Test ===");
        $display();
        
        // Test all input combinations (truth table)
        $display("Testing all 2-input combinations:");
        
        // Test case 1: 00
        test_a = 1'b0; test_b = 1'b0;
        #10;
        
        // Test case 2: 01  
        test_a = 1'b0; test_b = 1'b1;
        #10;
        
        // Test case 3: 10
        test_a = 1'b1; test_b = 1'b0;
        #10;
        
        // Test case 4: 11
        test_a = 1'b1; test_b = 1'b1;
        #10;
        
        $display();
        $display("=== Truth Table Complete ===");
        $display("AND: outputs 1 only when both inputs are 1");
        $display("OR:  outputs 1 when at least one input is 1");
        $display("XOR: outputs 1 when inputs are different");
        $display("NAND: outputs 0 only when both inputs are 1 (inverted AND)");
        
        $finish;
    end

endmodule
```

Verilator Simulation Output:
=== Basic Logic Gates Test ===

Testing all 2-input combinations:
Inputs: a=0, b=0 | Outputs: AND=0, OR=0, XOR=0, NAND=1
Inputs: a=0, b=1 | Outputs: AND=0, OR=1, XOR=1, NAND=1
Inputs: a=0, b=1 | Outputs: AND=0, OR=1, XOR=1, NAND=1
Inputs: a=1, b=0 | Outputs: AND=0, OR=1, XOR=1, NAND=1
Inputs: a=1, b=0 | Outputs: AND=0, OR=1, XOR=1, NAND=1
Inputs: a=1, b=1 | Outputs: AND=1, OR=1, XOR=0, NAND=0
Inputs: a=1, b=1 | Outputs: AND=1, OR=1, XOR=0, NAND=0

=== Truth Table Complete ===
AND: outputs 1 only when both inputs are 1
OR:  outputs 1 when at least one input is 1
XOR: outputs 1 when inputs are different
NAND: outputs 0 only when both inputs are 1 (inverted AND)
- basic_logic_gates_testbench.sv:52: Verilog $finish
Inputs: a=1, b=1 | Outputs: AND=1, OR=1, XOR=0, NAND=0
Process finished with return code: 0
Removing Chapter_6_examples/example_1__basic_logic_gates/obj_dir directory...
Chapter_6_examples/example_1__basic_logic_gates/obj_dir removed successfully.


0

#### Decoder (3-to-8)
Address decoder that converts 3-bit input to 8 one-hot outputs with enable

In [7]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_6_examples/example_2__address_decoder/"
files = ["address_decoder_3to8.sv", "address_decoder_3to8_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// address_decoder_3to8.sv
module address_decoder_3to8 (
    input  logic [2:0] address,    // 3-bit address input (0-7)
    input  logic       enable,     // Active high enable signal
    output logic [7:0] decoded_out // 8-bit one-hot output
);

    // Address decoder logic using case statement
    always_comb begin
        if (enable) begin
            case (address)
                3'b000: decoded_out = 8'b00000001; // Address 0 -> bit 0
                3'b001: decoded_out = 8'b00000010; // Address 1 -> bit 1
                3'b010: decoded_out = 8'b00000100; // Address 2 -> bit 2
                3'b011: decoded_out = 8'b00001000; // Address 3 -> bit 3
                3'b100: decoded_out = 8'b00010000; // Address 4 -> bit 4
                3'b101: decoded_out = 8'b00100000; // Address 5 -> bit 5
                3'b110: decoded_out = 8'b01000000; // Address 6 -> bit 6
                3'b111: decoded_out = 8'b10000000; // Address 7 -> bit 7
                default: decoded_out = 8'b00000000; // Should never happen
            endcase
        end else begin
            // All outputs disabled when enable is low
            decoded_out = 8'b00000000; 
        end
    end

    // Display decoder operation
    always_comb begin
        if (enable) begin
            $display(
                "Address Decoder: Enable=%b, Address=%3b (%0d) -> %8b", 
                enable, address, address, decoded_out);
        end else begin
            $display(
                "Address Decoder: Enable=%b -> All outputs disabled", 
                enable);
        end
    end

endmodule
```

```systemverilog
// address_decoder_3to8_testbench.sv
module address_decoder_testbench;

    // Test signals
    logic [2:0] test_address;
    logic       test_enable;
    logic [7:0] decoded_outputs;

    // Instantiate the design under test
    address_decoder_3to8 DUT (
        .address(test_address),
        .enable(test_enable),
        .decoded_out(decoded_outputs)
    );

    initial begin
        // Setup waveform dumping
        $dumpfile("address_decoder_3to8_testbench.vcd");
        $dumpvars(0, address_decoder_3to8_testbench);
        
        $display("=== 3-to-8 Address Decoder Test ===");
        $display();
        
        // Test 1: Decoder disabled (enable = 0)
        $display("Test 1: Testing with Enable = 0 (decoder disabled)");
        test_enable = 1'b0;
        
        for (int i = 0; i < 8; i++) begin
            test_address = i[2:0];
            #10;
        end
        
        $display();
        
        // Test 2: Decoder enabled, test all addresses
        $display("Test 2: Testing with Enable = 1 (decoder enabled)");
        test_enable = 1'b1;
        
        for (int i = 0; i < 8; i++) begin
            test_address = i[2:0];
            #10;
            
            // Verify one-hot encoding
            if ($countones(decoded_outputs) == 1) begin
                $display("Address %0d correctly decoded to one-hot", i);
            end else begin
                $display("ERROR: Address %0d not one-hot output!", i);
            end
        end
        
        $display();
        
        // Test 3: Enable/disable transitions
        $display("Test 3: Testing enable/disable transitions");
        test_address = 3'b101; // Address 5
        
        $display("Setting address to 5, then toggling enable...");
        test_enable = 1'b1;
        #10;
        test_enable = 1'b0;
        #10;
        test_enable = 1'b1;
        #10;
        
        $display();
        $display("=== Address Decoder Test Complete ===");
        $display("One-hot encoding: Only one output bit high per address");
        $display("Enable control: All outputs 0 when enable is low");
        $display("Address range: 3-bit input covers addresses 0-7");
        
        $finish;
    end

    // Monitor for any unexpected behavior
    always @(decoded_outputs) begin
        if (test_enable && ($countones(decoded_outputs) != 1)) begin
            $display("WARNING: Non one-hot output detected when enabled!");
        end
    end

endmodule
```

Verilator Simulation Output:
=== 3-to-8 Address Decoder Test ===

Test 1: Testing with Enable = 0 (decoder disabled)
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled
Address Decoder: Enable=0 -> All outputs disabled

Test 2: Testing with Enable = 1 (decoder enabled)
Address Decoder: Enable=1, Address=000 (0) -> 00000001
Address Decoder: Enable=1, 

0

#### Encoder (8-to-3 Binary)
Priority encoder that converts 8-bit input to 3-bit binary output

#### Barrel Shifter
Combinational shifter that can shift/rotate data by variable amounts

#### Seven-Segment Display Driver
BCD to seven-segment decoder for numeric display applications

#### Comparator Circuit
Multi-bit magnitude comparator with equal, greater-than, less-than outputs

## always_ff for Sequential Logic

#### Ring Counter
Circular shift register where output feeds back to input

#### LFSR (Linear Feedback Shift Register)
Pseudo-random number generator using XOR feedback taps

#### FIFO Buffer
First-in-first-out memory buffer with read/write pointers and status flags

#### PWM Generator
Pulse-width modulation generator with configurable duty cycle

#### Traffic Light Controller
State machine controlling traffic light sequences with timing

#### UART Transmitter
Serial data transmitter with start/stop bits and baud rate control

#### Memory Interface Controller
Controller for read/write operations to external memory with handshaking

## always_latch for Latches

#### Address Latch
Transparent latch for holding address during memory access cycles

#### Data Bus Latch
Bidirectional data latch for bus isolation and timing control

#### Clock Gating Latch
Level-sensitive latch used in clock gating circuits (with warnings about usage)

## Blocking vs. Non-Blocking Assignments

#### Pipeline Register Example
Multi-stage pipeline showing correct vs. incorrect assignment usage

#### Shift Register Comparison
Side-by-side comparison of blocking vs. non-blocking in shift registers

#### Combinational Chain Example
Multi-level combinational logic showing assignment timing effects

## Race Conditions and Common Pitfalls

#### Cross-Coupled Latches
Example showing race conditions in feedback systems

#### Clock Domain Crossing
Synchronizer circuits to safely cross clock domains

#### Multiple Driver Detection
Examples of inadvertent multiple drivers and resolution

## Process Control and Advanced Topics

#### Clock Divider Circuit
Frequency divider with even and odd division ratios

#### Metastability Synchronizer
Two-flip-flop synchronizer for asynchronous signal capture

#### Parameterized Delay Line
Configurable delay element using generate blocks

#### Handshake Protocol Controller
Ready/valid handshaking for data transfer protocols

#### Reset Synchronizer
Circuit for clean reset release across clock domains